# Revised Multiple Max Subarray Algorithm

In [1]:
import algs
import numpy as np
from matplotlib.pyplot import plot

In [2]:
def close_to_changes(close):
    close = np.array(close)
    return close[1:] - close[:-1]
close = np.array([100, 113, 110, 85, 105, 102, 86, 63, 81, 101, 94, 106, 101, 79, 94, 90, 97])
A = close_to_changes(close)

At each iteration $i$, we need to check $2i-1$ total regions.  We need to check if we need to split the existing $i-1$ regions.  Then we need to check the the $i$ regions outside those regions.

$checks=\sum_{i=1}^{n}{2i-1}=n^2$

In [3]:
class Subarray:
    def __init__(self, start, end, sum_):
        self.start = start
        self.end = end
        self.sum = sum_
        
    def __repr__(self):
        return f'[{self.start}, {self.end}]'
    
    def __str__(self):
        return f'[{self.start}, {self.end}]\tsum={self.sum}'

def max_sub(A, start=None, end=None):
    if start == None: start = 0
    if end == None: end = len(A)-1
    s, e, sm = algs.find_maximum_subarray(A, start, end, return_type=tuple)
    return Subarray(s, e, sm)

def print_subarrays(A, subarrays):
    last = 0
    for i in range(len(A)):
        print(f'{i:4d}', end='')
    print()
    for subarray in subarrays:
        for _ in range(subarray.start - last):
                print('    ', end='')
        if subarray.start == subarray.end:
            print('  []', end='')
        else:
            print('   [', end='')
            for _ in range(subarray.end - subarray.start - 1):
                print('    ', end='')
            print('   ]', end='')
        last = subarray.end + 1
    print()
    for a in A:
        print(f'{a:4d}', end='')    
    print()

def print_subarrays2(A, subarrays):
    last = 0
    
    for subarray in subarrays:
        
        # print elements before subarray
        for i in range(last, subarray.start):
            print(f'{A[i]} ', end='')
            
        print('[', end='')
        
        # print elements in subarray
        for i in range(subarray.start, subarray.end+1):
            print(A[i], end='')
            if i != subarray.end: print(end=' ')
        last = subarray.end
        
        print('] ', end='')
        
    # print elements past last subarray
    if last != len(A)-1:
        for i in range(last, len(A)):
            print(f'{A[i]} ', end='')
        print()

In [4]:
# rewrite 5/31
def multiple_max_sub(A, n_iter=None):
    
    # initialization
    subarrays = []
    n = len(A)
    if n_iter == None: n_iter = n
    
    # add a new subarray n_iter times
    for iteration in range(n_iter):

        result_max = Subarray(0, 0, -float('inf')) # keep track of max sum seen so far

        # split existing subarrays
        for i, subarray in enumerate(subarrays):
            start = subarray.start + 1
            end = subarray.end - 1
            if start > end: # existing subarray only has 1 or 2 elements, so you cannot split it further to increase the total sum
                continue

            result = max_sub(-A, start, end)
            if result.sum > result_max.sum:
                result_max = result
                ix_new_subarray = i
                split_existing = True


        # check regions outside existing subarrays
        # the number of outside regions is equal to the iteration+1 unless an existing subarray is at the beginning or end of the array
        for i in range(iteration+1):
            if i == 0:
                start = 0
            elif subarrays[i-1].end == n-1: # existing subarray borders array end, no region to check
                continue
            else:
                start = subarrays[i-1].end+1

            if i == iteration:
                end = n - 1
            elif subarrays[i].start == 0: # existing subarray borders array start, no region to check
                continue
            else:
                end = subarrays[i].start - 1

            result = max_sub(A, start, end)
            if result.sum > result_max.sum:
                result_max = result
                ix_new_subarray = i
                split_existing = False

        if result_max.sum <= 0:
            print(f'Optimal configuration found with {iteration} subarrays.')
            break

        # add new subarray(s) to existing subarrays
        if split_existing:
            # split existing subarray
            # remove old subarray and add two new subarrays that don't include the excluded region
            old_subarray = subarrays.pop(ix_new_subarray)

            start_left = old_subarray.start
            end_left = result_max.start-1
            sum_left = sum(A[start_left:end_left+1])

            start_right = result_max.end+1
            end_right = old_subarray.end
            sum_right = sum(A[start_right:end_right+1])

            subarrays.insert(ix_new_subarray, Subarray(start_left, end_left, sum_left))
            subarrays.insert(ix_new_subarray+1, Subarray(start_right, end_right, sum_right))

        else:
            subarrays.insert(ix_new_subarray, Subarray(result_max.start, result_max.end, result_max.sum))
    
    total_sum = 0
    for subarray in subarrays:
        total_sum += subarray.sum
    
    return subarrays, total_sum

In [5]:
n = len(A)
subarrays, total_sum = multiple_max_sub(A, 2)
print_subarrays(A, subarrays)
print_subarrays2(A, subarrays)
print(f'total sum: {total_sum}')

   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15
              []               [           ]
  13  -3 -25  20  -3 -16 -23  18  20  -7  12  -5 -22  15  -4   7
13 -3 -25 [20] 20 -3 -16 -23 [18 20 -7 12] 12 -5 -22 15 -4 7 
total sum: 63
